In [1]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
from timeit import default_timer as timer
from datetime import datetime, timedelta

importlib.reload(funcs)

<module 'funcs' from 'C:\\Users\\Mag\\Documents\\_projekty\\praca\\funcs.py'>

## 0. Wczytanie danych zrzuconych z bazy SQL do csv

In [2]:
politicians = pd.read_csv("00_dane/portraits.csv", sep='|')
#speeches = pd.read_csv("00_dane/speech_data.csv", sep='|')

#wczytujemy plik po modyfikacjach w SQL
speeches = pd.read_csv("00_dane/speeches.csv", sep='|')

C:\Users\Mag\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1. Znalezienie imienia i nazwiska przemawiającego

In [5]:
speeches['author'] = speeches['speech_raw'].apply(funcs.name_finder)

Zapisanie rozszerzonych danych do dalszych analiz.

In [5]:
speeches.to_csv('00_dane/speeches.csv', encoding='UTF=8')

## 2. Czyszczenie tekstu przemówień pt.1


### 2.1. Wczytanie danych po czyszczeniu SQL i dodaniu autora

In [49]:
politicians = pd.read_csv("00_dane/portraits.csv", sep='|')
speeches = pd.read_csv("00_dane/speeches_2.csv", sep='|')

#speeches.to_csv('00_dane/speeches_2.csv', encoding='UTF=8')

C:\Users\Mag\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
len(speeches['speech_raw']), sum(1-pd.isnull(speeches['author_final']))


(272221, 225388)

### 2.2. Czyszczenie znaków specjalnych i wycinanie nagłówka.

In [51]:
start = timer()
speeches['speech_raw'] = speeches['speech_raw'].apply(funcs.rem_head)
speeches['speech_raw'] = speeches['speech_raw'].apply(funcs.clear_text)
speeches['speech_title'] = speeches['speech_title'].apply(funcs.clear_text)
end = timer()
print(end - start)

190.62840582558692


Wycinam fragment, który jest tytułem:
1. Dodaję "\" przed nawiasami w tytule.
2. Wycinam tytuł z treści.

In [60]:
start = timer()
speeches['speech_title'] = speeches['speech_title'].apply(lambda t: re.sub('\(','\(',t))
speeches['speech_title'] = speeches['speech_title'].apply(lambda t: re.sub('\)','\)',t))
print(timedelta(seconds=timer()-start))

0:00:02.197919


Jeden "problematyczny" case.

In [116]:
speeches['speech_title'] = [re.sub('\[COM\\\\\(2005\\\\\)0119 - C6-0099/2005 - 2005/0043\\\\\(COD\\\\\)\]','',t) for t in speeches['speech_title']]
speeches['speech_raw'] = speeches['speech_raw'].apply(funcs.rem_nasty) 

In [118]:
speeches['speech_raw'] = [funcs.rem_title(t, r) for t, r, in zip(speeches['speech_title'], speeches['speech_raw'])]


### 2.3. Zapisanie zmodyfikowanego pliku

In [119]:
start = timer()
speeches.to_csv('00_dane/speeches_2.csv', encoding='UTF-8', index=False, sep="|")
print(timedelta(seconds=timer()-start))

0:00:29.467977


## 3. Czyszczenie tekstu przemówienia pt 3.

### 3.1. Wczytanie danych

In [46]:
politicians = pd.read_csv("00_dane/portraits.csv", sep='|')
speeches = pd.read_csv("00_dane/speeches_2.csv", sep='|')

C:\Users\Mag\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 3.2. Usuwam problematyczny wiersz z pustą wartością

In [61]:
speeches = speeches[~speeches['speech_raw'].isnull()]

### 3.3. Usuwanie przeszkadzających elementów, tj.:
- Wypowiedzi innych posłów i komentarze typu "Oklaski" pojawiające się w nawiasach
- "Poseł Imię Nazwisko:"
- Panie Marszałku!
- Pani Marszałek!
- Wysoka Izbo!
- Panie Ministrze!
- Dziękuję bardzo.

In [48]:
start = timer()
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('\([\w.,: ]+\)','',t))
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('[Pp]oseł ([\w -\.]+):','',t))
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('Panie Marszałku!','',t))
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('Pani Marszałek!','',t))
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('Wysoka Izbo!','',t))
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('Panie Ministrze!','',t))
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('Dziękuję bardzo.','',t))
speeches['speech_raw'] = speeches['speech_raw'].apply(lambda t: re.sub('\s+',' ',t))
print(timedelta(seconds=timer()-start))

0:01:12.245097


### 3.4. Zapisanie pliku

In [49]:
start = timer()
speeches.to_csv('00_dane/speeches_2.csv', encoding='UTF-8', index=False, sep="|")
print(timedelta(seconds=timer()-start))

0:00:31.695712


## 4. Łączenie fragmentów przemówień w całość

### 4.1. Wczytanie plików

In [51]:
politicians = pd.read_csv("00_dane/portraits.csv", sep='|')
speeches = pd.read_csv("00_dane/speeches_2.csv", sep='|')
len(speeches['speech_raw']), sum(1-pd.isnull(speeches['author_final']))

C:\Users\Mag\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(272220, 225387)

In [63]:
speeches = speeches[~speeches['speech_raw'].isnull()]

In [64]:
len(speeches['speech_raw']), sum(1-pd.isnull(speeches['author_final']))

(272217, 225385)

### 4.2. Łączenie

In [123]:
speeches = speeches.reset_index()


In [ ]:
start = timer()

dt, auth, t, sp = None, None, None, ''
sp_joined = pd.DataFrame()

for i in range(len(speeches.speech_raw)):
    dt_, auth_, t_, sp_ = speeches[['date_', 'author_final','speech_title', 'speech_raw']].loc[i]
    if (dt_!=dt and auth_!=auth and t_!=t and i!=0):
        sp_joined = pd.concat([sp_joined, pd.DataFrame([[auth, dt, t, sp]])])
        dt, auth, t, sp = dt_, auth_, t_ , sp_
    else:
        dt, auth, t = dt_, auth_, t_
        sp = ' '.join([sp, sp_])
    #print(dt_, id_p_, t_)
sp_joined = pd.concat([sp_joined, pd.DataFrame([[auth, dt, t, sp]])])
    
sp_joined.columns = ['author', 'date_', 'title', 'speech']

print(timedelta(seconds=timer()-start))

In [114]:
start = timer()
speeches.to_csv('00_dane/speeches_2.csv', encoding='UTF-8', index=False, sep="|")
sp_joined.to_csv('00_dane/sp_joined.csv', encoding='UTF-8', index=False, sep="|")
print(timedelta(seconds=timer()-start))

False

In [59]:
print(speeches['speech_raw'].iloc[241132])
print(re.sub('\([\w.,: ]+\)','XXXXXXXXXXXXX',speeches['speech_raw'].iloc[241132]))

 Spróbowałbym może odpowiedzieć najpierw panu posłowi Dolacie, który domaga się konkretów, w ogóle całej tej prawej stronie sali, która - zdaje się - politykę zagraniczną utożsamia z werbalnym awanturnictwem. Chciałbym przypomnieć jeden... ( Fakty, fakty.) ( Powiedział, że tylko jeden.) ...konkretny przykład skuteczności działania pana ministra Sikorskiego, którego to przykładu państwo być może nie znacie, bo nie chcecie znać. Chciałbym przypomnieć ubiegłoroczną, lipcową wizytę pana ministra w Republice Mołdowy w sytuacji, jaka w tym kraju panowała, olbrzymiego napięcia spowodowanego fałszerstwami przedwyborczymi... (Gwar na sali) Gdybyście państwo zechcieli mnie wysłuchać, to dowiedzielibyście się czegoś, czego, zdaje się, nie wiecie. ( Panie pośle, słuchamy, słuchamy.) ...grożącego wybuchem protestów na dużą skalę, rozlewem krwi. Ta wizyta była wizytą nie tylko polskiego ministra spraw zagranicznych, ale też wysokiej rangi przedstawiciela społeczności międzynarodowej, ponieważ pan mi